## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
import os
os.chdir('..')

from config import g_key
#print(g_key)
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Kaitangata,-46.2817,169.8464,48.99,79,0,4.00,NZ,clear sky
1,1,Mahebourg,-20.4081,57.7000,80.60,74,40,11.50,MU,scattered clouds
2,2,Seoul,37.5683,126.9778,32.00,93,100,3.44,KR,mist
3,3,Aklavik,68.2191,-135.0107,-25.60,68,20,12.66,CA,snow
4,4,Ushuaia,-54.8000,-68.3000,48.20,87,90,13.80,AR,drizzle


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria # Min 10; Max 50
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Kaitangata,-46.2817,169.8464,48.99,79,0,4.00,NZ,clear sky
2,2,Seoul,37.5683,126.9778,32.00,93,100,3.44,KR,mist
4,4,Ushuaia,-54.8000,-68.3000,48.20,87,90,13.80,AR,drizzle
10,10,Palmer,42.1584,-72.3287,30.99,54,75,5.75,US,broken clouds
11,11,Hamilton,39.1834,-84.5333,26.01,63,90,13.80,US,overcast clouds
20,20,San Patricio,28.0170,-97.5169,42.80,93,90,14.97,US,overcast clouds
26,26,Sorland,67.6670,12.6934,37.40,70,75,32.21,NO,broken clouds
34,34,Bethel,41.3712,-73.4140,30.99,54,90,8.05,US,overcast clouds
35,35,Albany,42.6001,-73.9662,28.99,42,83,1.01,US,broken clouds
36,36,Fortuna,40.5982,-124.1573,50.00,93,75,24.16,US,light rain


In [5]:
print("Verification for the max " + str(preferred_cities_df["Max Temp"].max()))
print("Verification for the min " + str(preferred_cities_df["Max Temp"].min()))

Verification for the max 50.0
Verification for the min 10.0


In [6]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.isna().sum())
print("---")
print(preferred_cities_df.isna().count())
print("---")
print(preferred_cities_df.count())
print("---")
print(preferred_cities_df.head(2))

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64
---
City_ID                155
City                   155
Lat                    155
Lng                    155
Max Temp               155
Humidity               155
Cloudiness             155
Wind Speed             155
Country                155
Current Description    155
dtype: int64
---
City_ID                155
City                   155
Lat                    155
Lng                    155
Max Temp               155
Humidity               155
Cloudiness             155
Wind Speed             155
Country                155
Current Description    155
dtype: int64
---
   City_ID        City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0  Kaitangata -46.2817  169.8464     48.99        79           0   
2    

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].dropna().copy()
clean_df["Hotel Name"] = ""
clean_df.head(10)
#clean_df.count()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kaitangata,NZ,48.99,-46.2817,169.8464,
2,Seoul,KR,32.00,37.5683,126.9778,
4,Ushuaia,AR,48.20,-54.8000,-68.3000,
10,Palmer,US,30.99,42.1584,-72.3287,
11,Hamilton,US,26.01,39.1834,-84.5333,
20,San Patricio,US,42.80,28.0170,-97.5169,
26,Sorland,NO,37.40,67.6670,12.6934,
34,Bethel,US,30.99,41.3712,-73.4140,
35,Albany,US,28.99,42.6001,-73.9662,
36,Fortuna,US,50.00,40.5982,-124.1573,


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kaitangata,NZ,48.99,clear sky,-46.2817,169.8464,
2,Seoul,KR,32.00,mist,37.5683,126.9778,
4,Ushuaia,AR,48.20,drizzle,-54.8000,-68.3000,
10,Palmer,US,30.99,broken clouds,42.1584,-72.3287,
11,Hamilton,US,26.01,overcast clouds,39.1834,-84.5333,
20,San Patricio,US,42.80,overcast clouds,28.0170,-97.5169,
26,Sorland,NO,37.40,broken clouds,67.6670,12.6934,
34,Bethel,US,30.99,overcast clouds,41.3712,-73.4140,
35,Albany,US,28.99,broken clouds,42.6001,-73.9662,
36,Fortuna,US,50.00,light rain,40.5982,-124.1573,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = pd.DataFrame(hotel_df)
#clean_hotel_df.isna().sum()
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kaitangata,NZ,48.99,clear sky,-46.2817,169.8464,Kaitangata Motor Camp
2,Seoul,KR,32.00,mist,37.5683,126.9778,Lotte Hotel Seoul
4,Ushuaia,AR,48.20,drizzle,-54.8000,-68.3000,Albatros Hotel
10,Palmer,US,30.99,broken clouds,42.1584,-72.3287,Wedgewood Motel
11,Hamilton,US,26.01,overcast clouds,39.1834,-84.5333,Six Acres Bed & Breakfast
20,San Patricio,US,42.80,overcast clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
26,Sorland,NO,37.40,broken clouds,67.6670,12.6934,Sjybrygga fiskecamp
34,Bethel,US,30.99,overcast clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
36,Fortuna,US,50.00,light rain,40.5982,-124.1573,Super 8 by Wyndham Fortuna
38,Juegang,CN,50.00,broken clouds,32.3174,121.1855,Rudong Y Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Description</b></dt><dd>{Current Description}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))